# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL
from pyspark.sql import types as T

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "mtecbo"
extrator_bronze = AlinharETL(bucket,datamart)

2024-10-15 10:28:13,667 - INFO - Iniciando Sessão Spark.


# Parâmetros para leitura do arquivo

In [3]:
schema = T.StructType([
    T.StructField("CODIGO", T.StringType(), True),
    T.StructField("TITULO", T.StringType(), True)
])

In [4]:
ano_mes = extrator_bronze.encontrar_ano_mes("mte/cbo")
local = f"s3a://landing/mte/cbo/{ano_mes}/CBO2002 - SubGrupo Principal.csv"

In [5]:
config_adicional = {
    'header': True,
    'sep': ';',
    'encoding': 'iso-8859-1'
}

# Leitura do arquivo

In [6]:
df_compliance_cbo_sub_grupo_principal = extrator_bronze.criar_view_temporaria_arquivo(local,'compliance_cbo_sub_grupo_principal',schema,**config_adicional)

2024-10-15 10:28:19,225 - INFO - Lista de yaml carregada com sucesso
2024-10-15 10:28:23,999 - INFO - Criando view temporaria.


In [7]:
sql_query = """
    SELECT CODIGO as cd,
           TITULO,
           SUBSTRING(codigo, 1, 1) AS cd_grande_grupo
    FROM compliance_cbo_sub_grupo_principal
"""

In [8]:
df_compliance_cbo_sub_grupo_principal = extrator_bronze.carregar_dados_delta(sql_query)

2024-10-15 10:28:24,013 - INFO - Aguarde... Executando Query (Delta)
2024-10-15 10:28:24,014 - INFO - 
    SELECT CODIGO as cd,
           TITULO,
           SUBSTRING(codigo, 1, 1) AS cd_grande_grupo
    FROM compliance_cbo_sub_grupo_principal

2024-10-15 10:28:24,242 - INFO - Query (Delta) executada com sucesso


# Gravação no datalake

In [9]:
extrator_bronze.caminho_tabela_delta = 's3a://bronze/mte/cbosubgrupoprincipal'

In [10]:
extrator_bronze.salvar_delta(df_compliance_cbo_sub_grupo_principal, 'overwrite')

2024-10-15 10:28:24,253 - INFO - Aguarde... Persistindo dados (overwrite)
2024-10-15 10:28:45,662 - INFO - Dados persistidos com sucesso
2024-10-15 10:28:45,663 - INFO - s3a://bronze/mte/cbosubgrupoprincipal
2024-10-15 10:28:45,664 - INFO - Aguarde... Realizando optimize
2024-10-15 10:28:55,366 - INFO - Optimize executado com sucesso.
2024-10-15 10:28:55,368 - INFO - Aguarde... Realizando vacuum
2024-10-15 10:29:14,920 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [11]:
extrator_bronze.parar_sessao()

2024-10-15 10:29:15,384 - INFO - Sessão Spark finalizada.
